In [1]:
from APICall import CompanyDetails
from neuralprophet import NeuralProphet, set_log_level, save
import pandas as pd
import json
import matplotlib.pyplot as plt

Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.


In [2]:
c_list = []
with open('companies_list.json', 'r') as file:
    companies = json.load(file)

count = 1

for company in companies: 
    c_list.append(company['c_symbol'])
    print(f'{count} : {company} : added to list')
    count += 1

1 : {'c_id': 1, 'c_symbol': 'RELIANCE', 'c_name': 'Reliance Industries Limited', 'marketcap': 201056022} : added to list
2 : {'c_id': 2, 'c_symbol': 'TCS', 'c_name': 'Tata Consultancy Services Limited', 'marketcap': 140247926} : added to list
3 : {'c_id': 3, 'c_symbol': 'HDFCBANK', 'c_name': 'HDFC Bank Limited', 'marketcap': 109991524} : added to list
4 : {'c_id': 4, 'c_symbol': 'ICICIBANK', 'c_name': 'ICICI Bank Limited', 'marketcap': 76765685} : added to list
5 : {'c_id': 5, 'c_symbol': 'BHARTIARTL', 'c_name': 'Bharti Airtel Limited', 'marketcap': 69478400} : added to list
6 : {'c_id': 6, 'c_symbol': 'SBIN', 'c_name': 'State Bank of India', 'marketcap': 67144318} : added to list
7 : {'c_id': 7, 'c_symbol': 'INFY', 'c_name': 'Infosys Limited', 'marketcap': 62179774} : added to list
8 : {'c_id': 8, 'c_symbol': 'LICI', 'c_name': 'Life Insurance Corporation Of India', 'marketcap': 57946466} : added to list
9 : {'c_id': 9, 'c_symbol': 'ITC', 'c_name': 'ITC Limited', 'marketcap': 53464378}

In [5]:
def sharePriceToDDF(c_list):
    share_price_df = {}

    for c_name in c_list:
        try:
            details = CompanyDetails(c_name)
            share_price = details.sharePriceRange(period='6y')
            df = pd.DataFrame({
                'ds' : pd.to_datetime(share_price['Date']),
                'y' : share_price['Close'],
                'ID' : c_name
            })
            share_price_df[c_name] = df
            print(f'{c_name} : added to the dict')
        except Exception as e:
            print(f'{c_name} : error company :{e}')

    return share_price_df


In [6]:
share_price_df = sharePriceToDDF(c_list)

RELIANCE : added to the dict
TCS : added to the dict
HDFCBANK : added to the dict
ICICIBANK : added to the dict
BHARTIARTL : added to the dict
SBIN : added to the dict
INFY : added to the dict
LICI : added to the dict
ITC : added to the dict
HINDUNILVR : added to the dict
LT : added to the dict
BAJFINANCE : added to the dict
HCLTECH : added to the dict
MARUTI : added to the dict
SUNPHARMA : added to the dict
ADANIENT : added to the dict
KOTAKBANK : added to the dict
TITAN : added to the dict
ONGC : added to the dict
TATAMOTORS : added to the dict
NTPC : added to the dict
AXISBANK : added to the dict
DMART : added to the dict
ADANIGREEN : added to the dict
ADANIPORTS : added to the dict
ULTRACEMCO : added to the dict
ASIANPAINT : added to the dict
COALINDIA : added to the dict
BAJAJFINSV : added to the dict
POWERGRID : added to the dict
WIPRO : added to the dict
IOC : added to the dict
JIOFIN : added to the dict
HAL : added to the dict
DLF : added to the dict
ADANIPOWER : added to the d

In [7]:
print(share_price_df.values())

dict_values([             ds        y        ID
0    2019-04-18   629.83  RELIANCE
1    2019-04-22   611.38  RELIANCE
2    2019-04-23   619.79  RELIANCE
3    2019-04-24   631.44  RELIANCE
4    2019-04-25   623.67  RELIANCE
...         ...      ...       ...
1477 2025-04-09  1185.35  RELIANCE
1478 2025-04-11  1218.95  RELIANCE
1479 2025-04-15  1240.10  RELIANCE
1480 2025-04-16  1239.30  RELIANCE
1481 2025-04-17  1274.50  RELIANCE

[1482 rows x 3 columns],              ds        y   ID
0    2019-04-18  1934.35  TCS
1    2019-04-22  1944.61  TCS
2    2019-04-23  1938.85  TCS
3    2019-04-24  1964.40  TCS
4    2019-04-25  1974.79  TCS
...         ...      ...  ...
1477 2025-04-09  3246.60  TCS
1478 2025-04-11  3231.50  TCS
1479 2025-04-15  3248.20  TCS
1480 2025-04-16  3273.80  TCS
1481 2025-04-17  3298.90  TCS

[1482 rows x 3 columns],              ds        y        ID
0    2019-04-18  1091.70  HDFCBANK
1    2019-04-22  1079.82  HDFCBANK
2    2019-04-23  1068.68  HDFCBANK
3    2019-04-24

In [13]:
df_all = pd.concat(share_price_df.values(), ignore_index=True)
df_all = (
    df_all
    .groupby('ID')
    .filter(lambda d: d['y'].nunique() > 1)
)
print(df_all)

               ds       y        ID
0      2019-04-18  629.83  RELIANCE
1      2019-04-22  611.38  RELIANCE
2      2019-04-23  619.79  RELIANCE
3      2019-04-24  631.44  RELIANCE
4      2019-04-25  623.67  RELIANCE
...           ...     ...       ...
645508 2025-04-09  577.90       ACI
645509 2025-04-11  601.10       ACI
645510 2025-04-15  613.25       ACI
645511 2025-04-16  633.20       ACI
645512 2025-04-17  638.65       ACI

[645512 rows x 3 columns]


In [18]:
# share_price_model = ''
set_log_level('ERROR')
share_price_model = NeuralProphet(
    normalize='minmax',      # or 'standardize'
    daily_seasonality=True
)

share_price_matrics = share_price_model.fit(df_all, freq='D')


WARNING - (py.warnings._showwarnmsg) - c:\Users\Aravind\AppData\Local\Programs\Python\Python39\lib\site-packages\neuralprophet\df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

WARNING - (py.warnings._showwarnmsg) - c:\Users\Aravind\AppData\Local\Programs\Python\Python39\lib\site-packages\neuralprophet\df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

WARNING - (py.warnings._showwarnmsg) - c:\Users\Aravind\AppData\Local\Programs\Python\Python39\lib\site-packages\neuralprophet\df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_d

Training: |          | 0/? [00:00<?, ?it/s]

Finding best initial lr:   0%|          | 0/295 [00:00<?, ?it/s]

WARNING - (py.warnings._showwarnmsg) - c:\Users\Aravind\AppData\Local\Programs\Python\Python39\lib\site-packages\pytorch_lightning\utilities\data.py:79: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 512. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.



Training: |          | 0/? [00:00<?, ?it/s]

WARNING - (py.warnings._showwarnmsg) - c:\Users\Aravind\AppData\Local\Programs\Python\Python39\lib\site-packages\pytorch_lightning\utilities\data.py:79: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 391. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.



In [20]:
save(share_price_model, 'globle_shareprice_model2.np')

In [19]:
save(share_price_model, 'Models/globle_shareprice_model.np')

KeyboardInterrupt: 